In [1]:
# universally modules
import re
import sys
import regex
import nltk.data
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from xgboost import XGBRegressor, XGBClassifier

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Parameters

In [2]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 100
# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

#### define the target variable and categorial variables used in later transformations ###
#### Case 1: gender 
target_variable = "gender"
categorial_variables =  ["topic", "sign"]

# Case 2: topic
#target_variable = "topic"
#categorial_variables =  ["gender", "sign"]

# Case 3: age
#target_variable = "age"
#categorial_variables =  ["topic", "gender", "sign"]

# Case 4: sign
#target_variable = "sign"
#categorial_variables =  ["gender", "topic"]
############################################################################################

# use only words that occur at least sqrt_3(X) times 
min_df_exponent = (1/3)

### Initialization 

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df = pd.read_csv("../resource/data/blogtext.csv")

In [4]:
# draw random sample for faster processing:
df = df.sample(sample_size)
df

,id,gender,age,topic,sign,date,text
251194,3610960,female,16,Student,Aries,"12,July,2004","they're long, complicated, nd som..."
653185,1015556,male,34,Technology,Virgo,"11,December,2003",Wow. Got some seriously cool...
524296,1960555,female,16,Student,Capricorn,"01,August,2004",Music: Foo Fighters x Everlong Mood:...
136724,4001907,male,27,Publishing,Aries,"23,July,2004",urlLink There have me and a c...
337212,3983169,male,17,indUnk,Aries,"22,July,2004",Blah.&nbsp; Its kinda early.&nbsp;...
...,...,...,...,...,...,...,...
43520,1713442,male,34,Education,Aquarius,"20,August,2003",urlLink Tales from the MSU Stacks I did a...
517506,180519,female,23,indUnk,Cancer,"28,March,2002",slipping into a funk again--my mind fee...
317127,1432868,male,16,Student,Scorpio,"15,September,2003",'Aoccdrnig to a rscheearch at an El...
61260,2871824,male,24,Engineering,Leo,"19,June,2004",Yesterday very busy until me always...


### Filtering

In [5]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [6]:
#def findDates(text):
#    try:
#        return len([date for date in\
#                    datefinder.find_dates(text)])
#    except:
#        return 0

In [7]:
def buildFeatures(text):
    text_split = text.split()
    len_text = len(text)
    sentence_split = tokenizer.tokenize(text)
    
    # find the number of urls in the text
    keywords = ["urlLink","http","www"]
    nb_urls = sum((any(keyword in pattern for keyword in keywords))\
               for pattern in text.split())
    # find the number of mails in the text
    nb_mails = len(re.findall(r"([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+|\bmail\b)"\
                      ,text))
    
    # find the number of dates in the text
#    nb_dates = findDates(text)
     
    # find characteristics about the usage of letters, numbers and symbols
    uppercase_ratio = len(re.findall(r'[A-Z]', text))/len_text
    lowercase_ratio = len(re.findall(r'[a-z]', text))/len_text
    number_ratio = len(re.findall(r'[0-9]', text))/len_text
    symbol_ratio = len(re.findall(r'[$-/:-?{-~!"^_`\[\]]', text))/len_text

    # find characteristics about the letters per word
    sentence_len_word = [len(word) for word in text_split]
    avg_letters_per_word = np.mean([len(word) for word in text_split])
    var_letters_per_word = np.var([len(word) for word in text_split])
    unique_words_ratio = len(set(text_split))/len(text_split)

    # find characteristics about the letters per sentence
    sentence_len_list = [len(sentence) for sentence in sentence_split]
    avg_letters_per_sentence = np.mean(sentence_len_list)
    var_letters_per_sentence = np.var(sentence_len_list)
    
    # find characteristics about the words per sentence
    words_per_sentence_len_list = [len(sentence.split()) for sentence in sentence_split]
    avg_words_per_sentence = np.mean(words_per_sentence_len_list)
    var_words_per_sentence = np.var(words_per_sentence_len_list)
    
    # find the trumps
    uppercase_per_sentence_ratio = [len(re.findall(r'[A-Z]', sentence))/len(sentence)\
                                    for sentence in sentence_split]
    max_sentence_uppercase_ratio = max(uppercase_per_sentence_ratio)
    max_sentence_uppercase_len = len(sentence_split[uppercase_per_sentence_ratio.index(max_sentence_uppercase_ratio)])
    
    return len_text, nb_urls, nb_mails,\
           uppercase_ratio, lowercase_ratio, number_ratio, symbol_ratio,\
           avg_letters_per_word, var_letters_per_word, unique_words_ratio,\
           avg_letters_per_sentence, var_letters_per_sentence,\
           avg_words_per_sentence, var_words_per_sentence,\
           max_sentence_uppercase_ratio, max_sentence_uppercase_len        
           

In [8]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# append the data
columns = ["Text length", "Number URLs", "Number mails",\
          "Uppercase ratio", "Lowercase ratio", "Number ratio", "Symbol ratio",\
          "Average letters per word", "Variance of letters per word", "Unique words ratio",\
          "Average letters per sentence", "Variance of letters per sentence",\
          "Average words per sentence", "Variance of words per sentence",\
          "Maximal uppercase ratio per sentence", "Length of the maximal uppercase ratio sentence"]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features, columns=columns), left_index=True, right_index=True)

100%|██████████| 95/95 [00:00<00:00, 800.30it/s]


### Text Preprocessing

In [9]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in tqdm(df_preprocessed["text_preprocessed"])]

100%|██████████| 95/95 [00:00<00:00, 8004.40it/s]


In [10]:
def remove_non_ascii_words(text):
    return "".join([f"{word} " for word in text.split() if len(re.findall(r'[^\x00-\x7f]', word)) == 0])


In [11]:
# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

# remove all words with non-ascii chars
df_filtered["text_preprocessed"] = df_filtered["text_preprocessed"].apply(remove_non_ascii_words)

### Transformation

In [27]:
class StackedTransformation:
    
    def  __init__(self, X=None, y=None, numerical_transformer=None, text_transformer=None, text_features=None):
        self.X = X
        self.y = y
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        self.text_features = text_features
        
    def data_split(self, X, y, test_size= 0.2):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def build_transformer(self, X=None, y=None,\
                          numerical_transformer=None, text_transformer=None, text_features=None):
        # check if variables are present
        if X == None:
            X = self.X
        if y == None:
            y = self.y
        if numerical_transformer == None:
            numerical_transformer = self.numerical_transformer
        if text_transformer == None:
            text_transformer = self.text_transformer
        if text_features == None:
            text_features = self.text_features

        # split the data
        
        self.data_split(X, y)
        
        # Create datasets for each classifier
        self.X_train_text = self.X_train[text_features]
        self.X_test_text =  self.X_test[text_features]
        
        self.X_train_numerical = self.X_train.drop(text_features, axis=1)
        self.X_test_numerical = self.X_test.drop(text_features, axis=1)

        # create transformers 
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        
        #create transformed training batches
        self.X_train_numerical_transformed = self.numerical_transformer.fit_transform(self.X_train_numerical)
        self.X_test_numerical_transformed = self.numerical_transformer.transform(self.X_test_numerical)

        self.X_train_text_transformed = self.text_transformer.fit_transform(self.X_train_text)
        self.X_test_text_transformed = self.text_transformer.transform(self.X_test_text)

    def transform_one(self, x):
        entry = pd.DataFrame(columns=x.keys())
        entry = entry.append(x, ignore_index=True)
        
        # create datasets for each transformer
        entry_text = entry[text_features]
        entry_numerical = entry.drop(text_features, axis=1)
        
        entry_numerical_transformed = self.numerical_transformer.transform(entry_numerical)
        entry_text_transformed = self.text_transformer.transform(entry_text)
        
        return {"transformed_text": entry_text_transformed,\
               "transformed_numerical": entry_numerical_transformed}

    def transform_one(self, x):
        entry = pd.DataFrame(columns=x.keys())
        entry = entry.append(x, ignore_index=True)
        
        # create datasets for each transformer
        entry_text = entry[self.text_features]
        entry_numerical = entry.drop(self.text_features, axis=1)
        
        entry_numerical_transformed = self.numerical_transformer.transform(entry_numerical)
        entry_text_transformed = self.text_transformer.transform(entry_text)
        
        return {"transformed_text": entry_text_transformed,\
               "transformed_numerical": entry_numerical_transformed}


    def transform_many(self, X):
        # create datasets for each transformer
        X_text = X[self.text_features]
        X_numerical = X.drop(self.text_features, axis=1)
        
        X_numerical_transformed = self.numerical_transformer.transform(X_numerical)
        X_text_transformed = self.text_transformer.transform(X_text)
        
        return {"transformed_text": X_text_transformed,\
               "transformed_numerical": X_numerical_transformed}

### Data Split

In [28]:
X,y = df_filtered.drop(target_variable, axis=1),df_filtered[target_variable]

In [29]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(X)**(min_df_exponent)))
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), categorial_variables)\
                                                       , remainder=StandardScaler())

stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
stacking.build_transformer()

### Clustering

In [16]:
text_data_features = stacking.text_transformer.get_feature_names()
text_data = stacking.X_train_text_transformed.toarray()

df_text_cluster = pd.DataFrame(text_data, columns=text_data_features)
df_text_cluster

# Die Features beschreiben die Worte im Text
# Die Werte sind die TF*IDF-transformierten Textdaten

,10,30,ask,back,bad,be,beautiful,bed,big,bit,...,wo,word,work,world,wow,write,yay,yeah,year,yesterday
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.084975,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.329005,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.044264,0.000000,0.035420,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.057623,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.154424,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.342544,0.486032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.239198,0.110719,0.000000,0.000000,0.000000,0.070616,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.232699,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
71,0.000000,0.000000,0.000000,0.000000,0.000000,0.124626,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.211073,0.000000,0.000000
72,0.061253,0.056705,0.064042,0.045196,0.047461,0.289328,0.064042,0.128085,0.0,0.000000,...,0.000000,0.0,0.119177,0.000000,0.0,0.000000,0.0,0.061253,0.000000,0.000000
73,0.000000,0.000000,0.250010,0.264655,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.125005,0.0,0.387706,0.000000,0.0,0.106962,0.0,0.000000,0.000000,0.131445


In [16]:
numerical_data = stacking.X_train_numerical_transformed
numerical_data_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                     stacking.X_train_numerical.columns.drop(categorial_variables))

df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_data_features)
df_numerical_cluster


# Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
# das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
# Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

,x0_Accounting,x0_Agriculture,x0_Arts,x0_Communications-Media,x0_Education,x0_Engineering,x0_Fashion,x0_Internet,x0_InvestmentBanking,x0_Law,...,Symbol ratio,Average letters per word,Variance of letters per word,Unique words ratio,Average letters per sentence,Variance of letters per sentence,Average words per sentence,Variance of words per sentence,Maximal uppercase ratio per sentence,Length of the maximal uppercase ratio sentence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.728211,-0.373913,-0.405588,-1.034518,-0.172704,-0.172048,-0.114345,-0.149941,-0.426713,-0.508643
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.441599,0.662686,-0.225826,0.630411,-0.519892,-0.226515,-0.509414,-0.208545,3.848010,-0.592957
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.215583,1.093433,-0.397563,1.570226,-0.562348,-0.256236,-0.608481,-0.254289,-0.491851,-0.068333
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.613411,-0.303795,-0.514383,0.885276,-0.344406,-0.201200,-0.292459,-0.192371,-0.337887,-0.424328
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.947118,1.279772,-0.381717,1.409860,2.008931,-0.266308,1.353031,-0.254289,0.001620,2.030163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.155387,0.424941,-0.135984,1.149267,-0.463039,-0.247183,-0.499508,-0.251388,-0.330291,0.081559
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.791656,0.483748,-0.272493,-0.283885,-0.004562,-0.168864,-0.053710,-0.159177,-0.193546,-0.218226
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.342296,-0.993565,-0.594833,1.206153,-0.020178,-0.214950,0.072600,-0.170452,-0.703552,-0.068333
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.252834,-0.861755,-0.403734,-1.769874,-0.340958,-0.184649,-0.229552,-0.154482,0.672502,-0.592957


### Training 

In [80]:
class StackedModelling:
    
    def __init__(self,  numerical_model, numerical_model_params,\
                text_model, text_model_params, stacked_transformation_instance,\
                weights=(0.5, 0.5)):
        
        # Initialize numerical model
        self.numerical_model = numerical_model(**numerical_model_params)
        
        # Initialize text model
        self.text_model = text_model(**text_model_params)
        
        # internalize transformation class
        self.stacked_transformation = stacked_transformation_instance
        
        # model weights
        self.weights = weights
        
    def fit(self):
        # train numerical model
        self.numerical_model.fit(self.stacked_transformation.X_train_numerical_transformed,\
                             self.stacked_transformation.y_train)
                             
        print("Numerical model finished!")
        
        # train textual model
        self.text_model.fit(self.stacked_transformation.X_train_text_transformed,\
                             self.stacked_transformation.y_train)
                             
        print("Text model finished!")
              
    def predict_text(self, X):
        return self.text_model.predict(X)
    
    def predict_numerical(self, X):
        return self.numerical_model.predict(X)

    
    def optimize_weights(self, X, y):
        # get loss for both models
        y_pred_numerical = self.numerical_model.predict(X)
        y_pred_textual = self.text_model.predict(X)
        
        
    
    def weighted_prediction(self, X, weights=None, algo_type="classification"):
        if weights == None:
            weights = self.weights
        
        # check if one or more transactions become processed
        try:
            if type(X) == dict:
                X_transformed = self.stacked_transformation.transform_one(X)

            else:
                X_transformed = self.stacked_transformation.transform_many(X)
        except TypeError:
            print("Check the data type of the input data")

        
        return (self.text_model.predict_proba(X_transformed["transformed_text"]),\
                self.numerical_model.predict_proba(X_transformed["transformed_numerical"]))


In [81]:
stacked_modelling = StackedModelling(XGBClassifier, {}, XGBClassifier, {}, stacking, (0.5, 0.5))
stacked_modelling.fit()
stacked_modelling.weighted_prediction(X)

Numerical model finished!
Numerical model finished!


(array([[0.79469717, 0.20530286],
        [0.03773683, 0.96226317],
        [0.9779954 , 0.02200458],
        [0.5052613 , 0.49473873],
        [0.39564717, 0.60435283],
        [0.9688676 , 0.03113239],
        [0.05333126, 0.94666874],
        [0.9268388 , 0.07316116],
        [0.17015117, 0.8298488 ],
        [0.05610365, 0.94389635],
        [0.6879061 , 0.3120939 ],
        [0.0286141 , 0.9713859 ],
        [0.08402741, 0.9159726 ],
        [0.02014595, 0.97985405],
        [0.04644632, 0.9535537 ],
        [0.03177005, 0.96822995],
        [0.03944409, 0.9605559 ],
        [0.10532731, 0.8946727 ],
        [0.03247142, 0.9675286 ],
        [0.5052613 , 0.49473873],
        [0.6597464 , 0.34025362],
        [0.6751555 , 0.32484445],
        [0.95875484, 0.04124515],
        [0.8984462 , 0.10155379],
        [0.04871005, 0.95128995],
        [0.9465293 , 0.05347068],
        [0.5261258 , 0.47387418],
        [0.03247142, 0.9675286 ],
        [0.94721663, 0.05278336],
        [0.030

In [78]:
test = XGBClassifier()

In [ ]:
test.

### Parameter Tuning 

### Model Evaluation 